<a href="https://www.kaggle.com/code/mahbuburrahman2020/se-europe-data-challenge-data-processing?scriptVersionId=161686379" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import argparse

In [ ]:
def load_data(file_path):
    # TODO: Load data from CSV file

    return df

In [ ]:
def clean_data(df):
    # TODO: Handle missing values, outliers, etc.

    return df_clean

In [ ]:
def preprocess_data(df):
    # TODO: Generate new features, transform existing features, resampling, etc.

    return df_processed

In [ ]:
def save_data(df, output_file):
    # TODO: Save processed data to a CSV file
    pass

In [ ]:
def parse_arguments():
    parser = argparse.ArgumentParser(description='Data processing script for Energy Forecasting Hackathon')
    parser.add_argument(
        '--input_file',
        type=str,
        default='data/raw_data.csv',
        help='Path to the raw data file to process'
    )
    parser.add_argument(
        '--output_file', 
        type=str, 
        default='data/processed_data.csv', 
        help='Path to save the processed data'
    )
    return parser.parse_args()

In [ ]:
def main(input_file, output_file):
    df = load_data(input_file)
    df_clean = clean_data(df)
    df_processed = preprocess_data(df_clean)
    save_data(df_processed, output_file)

In [ ]:
if __name__ == "__main__":
    args = parse_arguments()
    main(args.input_file, args.output_file)

**Data processing for energy generation data**

The processing of raw data is splitted into 3 parts:
* resample data into hourly level
* resample data into daily level
* resample data into monthly level

**Part 1:**

I do raw data processing to resample them into hourly level from 15, 30 or 45 min interval data. At first I load the generation data from different country folder. Each country folder contains csv files of different generation type.

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
file_name = "uk/gen_UK_B19.csv"

In [ ]:
df_gen = pd.read_csv(f"/kaggle/input/europe-green-electricity-generation-consumption/2022-2023/{file_name}")
df_gen

In [ ]:
df_gen.dtypes

In the following, I do data profiling for the loaded data to see data overview on that csv file.

In [ ]:
file_name = "gen_UK_B19_data_profile.html"

In [ ]:
from ydata_profiling import ProfileReport
df_gen_data_profile_path = os.path.join('/kaggle/working/', file_name)
profile = ProfileReport(df_gen, title="Energy Generation UK B19")
profile.to_file(df_gen_data_profile_path)

For the hourly level data, I dropped the area id, unit name, production or generation type columns data. 

I dropped the area id column data because this column data contain one area id for each country. Also this column contains some missing values for few countries. 

The reason to drop the unit name column data is the unit is same for all records and it's constant.

The data from the production or generation type column is dropped because from each file name I have the generation type code for specific country. So if I need the generation type code then I can use from the file name.

In [ ]:
df_gen = df_gen.drop(labels=["AreaID", "UnitName", "PsrType"], axis=1)
df_gen

I resample the minute level data into hourly level. In this case, the data varies in some files. There are few csv files which have hourly level data so I don't resample those data. Some csv files contain 30 minute interval data and some csv files contain 15 min interval data.

* data at a 15-minute resolution should be aggregated into 1-hour intervals by summing every 4 consecutive rows.
* data at a 30-minute resolution should be aggregated into 1-hour intervals by summing every 2 consecutive rows.

In [ ]:
from datetime import datetime, timedelta, timezone

df_gen_resampling = pd.DataFrame(data=[], columns=["StartTime", "EndTime", "quantity"])

#num_time_slot_in_hour = 4
num_time_slot_in_hour = 2
#interval_min_in_hour = 15
interval_min_in_hour = 30
interval_count = 0
quantity = 0
index = 0
indx = 0
start_date_1 = datetime.now()
start_date = datetime.now()
pre_end_date = datetime.now()
next_start_date = datetime.now()
interval_min = timedelta(minutes=interval_min_in_hour)
interval_hour = timedelta(hours=1)

while index < df_gen.shape[0]:
    row = df_gen.loc[index]
    index = index + 1
    if interval_count == num_time_slot_in_hour: 
        start_date_1 = datetime(start_date_1.year, start_date_1.month, start_date_1.day, start_date_1.hour, tzinfo=start_date_1.tzinfo)
        df_gen_resampling.loc[indx, "StartTime"] = start_date_1
        df_gen_resampling.loc[indx, "EndTime"] = start_date_1 + interval_hour
        df_gen_resampling.loc[indx, "quantity"] = quantity
        interval_count = 0
        quantity = 0
        indx = indx + 1
    
    start_date = datetime.fromisoformat(str(row["StartTime"]).replace("Z", ""))
    row_end_date = datetime.fromisoformat(str(row["EndTime"]).replace("Z", ""))
    end_date = start_date + interval_min
    
    if interval_count == 0:
        start_date_1 = start_date
        interval_count = interval_count + 1
        if end_date == row_end_date:
            quantity = quantity + row["quantity"]
    else:
        iter_count = 0
        while interval_count < num_time_slot_in_hour:
            if iter_count == 0:
                pre_end_date = datetime.fromisoformat(str(df_gen.loc[index - 2, "EndTime"]).replace("Z", ""))
            else:
                pre_end_date = pre_end_date + interval_min
            iter_count = iter_count + 1
            pre_start_date = pre_end_date - interval_min
            interval_count = interval_count + 1
            if start_date == pre_end_date and start_date.hour == pre_start_date.hour:
                if end_date > pre_end_date and end_date == row_end_date:
                    quantity = quantity + row["quantity"]
                    break
            if interval_count == num_time_slot_in_hour:
                index = index - 1

In [ ]:
start_date_1 = datetime(start_date_1.year, start_date_1.month, start_date_1.day, start_date_1.hour, tzinfo=start_date_1.tzinfo)
df_gen_resampling.loc[indx, "StartTime"] = start_date_1
df_gen_resampling.loc[indx, "EndTime"] = start_date_1 + interval_hour
df_gen_resampling.loc[indx, "quantity"] = quantity

In [ ]:
df_gen_resampling

In [ ]:
file_name = "gen_UK_B19_resampling.csv"
df_gen_resampling.to_csv(f"/kaggle/working/{file_name}", index=False)

**Part 2:**

I resample data into 1 day interval from the start hour of the day till end hour of the day. So for one year, if there is data for 365 days then after the execution of the following code, 365 rows are generated where the start time is the start time of the day and the end time is the 24-hour day's end time of the day.

In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
def is_file_exist(filedir, filename):
    file_exist = False
    #for _, _, filenames in os.walk('/kaggle/working'):
    for _, _, filenames in os.walk(filedir):
        if filename in filenames:
            file_exist = True
            break
    return file_exist

In [11]:
file_name = "uk/gen_UK_B19.csv"

In [12]:
df_gen = pd.read_csv(f"/kaggle/input/europe-green-electricity-generation-consumption/2022-2023/{file_name}")
df_gen

,StartTime,EndTime,AreaID,UnitName,PsrType,quantity
0,2022-01-26T23:30+00:00Z,2022-01-27T00:00+00:00Z,10Y1001A1001A92E,MAW,B19,433
1,2022-01-27T00:00+00:00Z,2022-01-27T00:30+00:00Z,10Y1001A1001A92E,MAW,B19,472
2,2022-01-27T00:30+00:00Z,2022-01-27T01:00+00:00Z,10Y1001A1001A92E,MAW,B19,549
3,2022-01-27T01:00+00:00Z,2022-01-27T01:30+00:00Z,10Y1001A1001A92E,MAW,B19,520
4,2022-01-27T01:30+00:00Z,2022-01-27T02:00+00:00Z,10Y1001A1001A92E,MAW,B19,407
...,...,...,...,...,...,...
12639,2022-12-31T21:00+00:00Z,2022-12-31T21:30+00:00Z,NaN,MAW,B19,183
12640,2022-12-31T21:30+00:00Z,2022-12-31T22:00+00:00Z,NaN,MAW,B19,155
12641,2022-12-31T22:00+00:00Z,2022-12-31T22:30+00:00Z,NaN,MAW,B19,132
12642,2022-12-31T22:30+00:00Z,2022-12-31T23:00+00:00Z,NaN,MAW,B19,112


In [13]:
from datetime import datetime, timedelta, timezone
start_date = datetime.fromisoformat(str(df_gen.loc[0, "StartTime"]).replace("Z", ""))
if start_date.year == 2021:
    df_gen = df_gen.drop(0)
    df_gen = df_gen.set_index(pd.Index(np.arange(0, df_gen.shape[0])))
df_gen

,StartTime,EndTime,AreaID,UnitName,PsrType,quantity
0,2022-01-26T23:30+00:00Z,2022-01-27T00:00+00:00Z,10Y1001A1001A92E,MAW,B19,433
1,2022-01-27T00:00+00:00Z,2022-01-27T00:30+00:00Z,10Y1001A1001A92E,MAW,B19,472
2,2022-01-27T00:30+00:00Z,2022-01-27T01:00+00:00Z,10Y1001A1001A92E,MAW,B19,549
3,2022-01-27T01:00+00:00Z,2022-01-27T01:30+00:00Z,10Y1001A1001A92E,MAW,B19,520
4,2022-01-27T01:30+00:00Z,2022-01-27T02:00+00:00Z,10Y1001A1001A92E,MAW,B19,407
...,...,...,...,...,...,...
12639,2022-12-31T21:00+00:00Z,2022-12-31T21:30+00:00Z,NaN,MAW,B19,183
12640,2022-12-31T21:30+00:00Z,2022-12-31T22:00+00:00Z,NaN,MAW,B19,155
12641,2022-12-31T22:00+00:00Z,2022-12-31T22:30+00:00Z,NaN,MAW,B19,132
12642,2022-12-31T22:30+00:00Z,2022-12-31T23:00+00:00Z,NaN,MAW,B19,112


* I sum all values of quantity column for the specific day from the start hour of that 24 hour's day to end of that 24 hour's day to calculate the total energy generation for that day.

* I add few new columns in the resampled dataset, the new columns are:
1. AreaID
2. PsrType
3. Country
4. CountryID
5. UnitName
6. GenType
7. PsrTypeName

PsrType: This is the energy generation type, there are 9 types of green energy generation, they are:

* B01 = Biomass
* B09 = Geothermal
* B10 = Hydro Pumped Storage
* B11 = Hydro Run-of-river and poundage
* B12 = Hydro Water Reservoir
* B13 = Marine
* B15 = Other renewable
* B16 = Solar
* B18 = Wind Offshore
* B19 = Wind Onshore

GenType Energy generation type which is one level up of the PsrType.

* B18 and B19 = Wind
* B16 = Solar
* B10, B11, B12, B13 = Water
* B01, B09, B15 = Other

AreaID: the areas of the energy generation.

Country: CountryID:

* SP: 0, # Spain
* UK: 1, # United Kingdom
* DE: 2, # Germany
* DK: 3, # Denmark
* HU: 5, # Hungary
* SE: 4, # Sweden
* IT: 6, # Italy
* PO: 7, # Poland
* NL: 8 # Netherlands

Unit Name: MAW(Magnetostrictive Amorphous Wire)

In [15]:
from datetime import datetime, timedelta, timezone
import calendar
country = "UK"
country_id = 1
#df_gen = df_gen.loc[1:]
file_exist = is_file_exist("/kaggle/input/europe-green-electricity-generation-consumption", "gen_2022_daily.csv")
if file_exist == True:
    df_gen_2022_daily = pd.read_csv("/kaggle/input/europe-green-electricity-generation-consumption/gen_2022_daily.csv")
    indx = df_gen_2022_daily.shape[0]
else:
    df_gen_2022_daily = pd.DataFrame(data=[], columns=["StartTime", "EndTime", "PsrType", "PsrTypeName", "GenType", "AreaID", "Country", "CountryID", "quantity", "UnitName"])
    indx = 0
index = 0
init_date = datetime.fromisoformat(str(df_gen.loc[0, "StartTime"]).replace("Z", ""))
day = init_date.day
start_hour = init_date.hour
start_min = init_date.minute
quantity = 0
start_date = datetime.now()
pre_start_date = datetime.now()
area_id = ""
while index < df_gen.shape[0]:
    row = df_gen.loc[index]
    index = index + 1
    start_date = datetime.fromisoformat(str(row["StartTime"]).replace("Z", ""))
    if start_date.hour == start_hour and start_date.minute == start_min:
        area_id = row["AreaID"]
    if start_date.day == day:
        quantity = quantity + row["quantity"]
    else:
        pre_start_date = datetime.fromisoformat(str(df_gen.loc[index - 2, "StartTime"]).replace("Z", ""))
        pre_start_date = datetime(pre_start_date.year, pre_start_date.month, pre_start_date.day, tzinfo=start_date.tzinfo)
        df_gen_2022_daily.loc[indx, "StartTime"] = pre_start_date
        interval_day = timedelta(hours=24)
        df_gen_2022_daily.loc[indx, "EndTime"] = pre_start_date + interval_day
        df_gen_2022_daily.loc[indx, "StartTime"] = df_gen_2022_daily.loc[indx, "StartTime"].date()
        df_gen_2022_daily.loc[indx, "EndTime"] = df_gen_2022_daily.loc[indx, "EndTime"].date()
        df_gen_2022_daily.loc[indx, "PsrType"] = row["PsrType"]
        df_gen_2022_daily.loc[indx, "AreaID"] = area_id
        df_gen_2022_daily.loc[indx, "Country"] = country
        df_gen_2022_daily.loc[indx, "CountryID"] = country_id
        df_gen_2022_daily.loc[indx, "quantity"] = quantity
        df_gen_2022_daily.loc[indx, "UnitName"] = row["UnitName"]
        
        if row["PsrType"] == "B18" or row["PsrType"] == "B19":
            df_gen_2022_daily.loc[indx, "GenType"] = "Wind"
        elif row["PsrType"] == "B16":
            df_gen_2022_daily.loc[indx, "GenType"] = "Solar"
        elif row["PsrType"] == "B10" or row["PsrType"] == "B11" or row["PsrType"] == "B12" or row["PsrType"] == "B13":
            df_gen_2022_daily.loc[indx, "GenType"] = "Water"
        elif row["PsrType"] == "B01" or row["PsrType"] == "B09" or row["PsrType"] == "B15":
            df_gen_2022_daily.loc[indx, "GenType"] = "Other"

        if row["PsrType"] == "B18":
            df_gen_2022_daily.loc[indx, "PsrTypeName"] = "Wind Offshore"
        elif row["PsrType"] == "B19":
            df_gen_2022_daily.loc[indx, "PsrTypeName"] = "Wind Onshore"
        elif row["PsrType"] == "B16":
            df_gen_2022_daily.loc[indx, "PsrTypeName"] = "Solar"
        elif row["PsrType"] == "B10":
            df_gen_2022_daily.loc[indx, "PsrTypeName"] = "Hydro Pumped Storage"
        elif row["PsrType"] == "B11":
            df_gen_2022_daily.loc[indx, "PsrTypeName"] = "Hydro Run-of-river and poundage"
        elif row["PsrType"] == "B12":
            df_gen_2022_daily.loc[indx, "PsrTypeName"] = "Hydro Water Reservoir"
        elif row["PsrType"] == "B01":
            df_gen_2022_daily.loc[indx, "PsrTypeName"] = "Biomass"
        elif row["PsrType"] == "B09":
            df_gen_2022_daily.loc[indx, "PsrTypeName"] = "Geothermal"
        elif row["PsrType"] == "B13":
            df_gen_2022_daily.loc[indx, "PsrTypeName"] = "Marine" 
        elif row["PsrType"] == "B15":
            df_gen_2022_daily.loc[indx, "PsrTypeName"] = "Other renewable"
        
        indx = indx + 1
        day = start_date.day
        next_row = datetime.fromisoformat(str(df_gen.loc[index, "StartTime"]).replace("Z", ""))
        start_hour = next_row.hour
        start_min = next_row.minute
        quantity = row["quantity"]

In [16]:
pre_start_date = datetime.fromisoformat(str(df_gen.loc[index - 2, "StartTime"]).replace("Z", ""))
pre_start_date = datetime(pre_start_date.year, pre_start_date.month, pre_start_date.day, tzinfo=start_date.tzinfo)
df_gen_2022_daily.loc[indx, "StartTime"] = pre_start_date
interval_day = timedelta(hours=24)
df_gen_2022_daily.loc[indx, "EndTime"] = pre_start_date + interval_day
df_gen_2022_daily.loc[indx, "StartTime"] = df_gen_2022_daily.loc[indx, "StartTime"].date()
df_gen_2022_daily.loc[indx, "EndTime"] = df_gen_2022_daily.loc[indx, "EndTime"].date()
df_gen_2022_daily.loc[indx, "PsrType"] = row["PsrType"]
df_gen_2022_daily.loc[indx, "AreaID"] = area_id
df_gen_2022_daily.loc[indx, "Country"] = country
df_gen_2022_daily.loc[indx, "CountryID"] = country_id
df_gen_2022_daily.loc[indx, "quantity"] = quantity
df_gen_2022_daily.loc[indx, "UnitName"] = row["UnitName"]

if row["PsrType"] == "B18" or row["PsrType"] == "B19":
    df_gen_2022_daily.loc[indx, "GenType"] = "Wind"
elif row["PsrType"] == "B16":
    df_gen_2022_daily.loc[indx, "GenType"] = "Solar"
elif row["PsrType"] == "B10" or row["PsrType"] == "B11" or row["PsrType"] == "B12" or row["PsrType"] == "B13":
    df_gen_2022_daily.loc[indx, "GenType"] = "Water"
elif row["PsrType"] == "B01" or row["PsrType"] == "B09" or row["PsrType"] == "B15":
    df_gen_2022_daily.loc[indx, "GenType"] = "Other"

if row["PsrType"] == "B18":
    df_gen_2022_daily.loc[indx, "PsrTypeName"] = "Wind Offshore"
elif row["PsrType"] == "B19":
    df_gen_2022_daily.loc[indx, "PsrTypeName"] = "Wind Onshore"
elif row["PsrType"] == "B16":
    df_gen_2022_daily.loc[indx, "PsrTypeName"] = "Solar"
elif row["PsrType"] == "B10":
    df_gen_2022_daily.loc[indx, "PsrTypeName"] = "Hydro Pumped Storage"
elif row["PsrType"] == "B11":
    df_gen_2022_daily.loc[indx, "PsrTypeName"] = "Hydro Run-of-river and poundage"
elif row["PsrType"] == "B12":
    df_gen_2022_daily.loc[indx, "PsrTypeName"] = "Hydro Water Reservoir"
elif row["PsrType"] == "B01":
    df_gen_2022_daily.loc[indx, "PsrTypeName"] = "Biomass"
elif row["PsrType"] == "B09":
    df_gen_2022_daily.loc[indx, "PsrTypeName"] = "Geothermal"
elif row["PsrType"] == "B13":
    df_gen_2022_daily.loc[indx, "PsrTypeName"] = "Marine"    
elif row["PsrType"] == "B15":
    df_gen_2022_daily.loc[indx, "PsrTypeName"] = "Other renewable"

In [17]:
df_gen_2022_daily

,StartTime,EndTime,PsrType,PsrTypeName,GenType,AreaID,Country,CountryID,quantity,UnitName
0,2022-01-01,2022-01-02,B01,Biomass,Other,10Y1001A1001A83F,DE,2.0,428653.0,MAW
1,2022-01-02,2022-01-03,B01,Biomass,Other,10Y1001A1001A83F,DE,2.0,427979.0,MAW
2,2022-01-03,2022-01-04,B01,Biomass,Other,10Y1001A1001A83F,DE,2.0,429196.0,MAW
3,2022-01-04,2022-01-05,B01,Biomass,Other,10Y1001A1001A83F,DE,2.0,432646.0,MAW
4,2022-01-05,2022-01-06,B01,Biomass,Other,10Y1001A1001A83F,DE,2.0,424438.0,MAW
...,...,...,...,...,...,...,...,...,...,...
21415,2022-12-27,2022-12-28,B19,Wind Onshore,Wind,NaN,UK,1.0,20744.0,MAW
21416,2022-12-28,2022-12-29,B19,Wind Onshore,Wind,NaN,UK,1.0,19062.0,MAW
21417,2022-12-29,2022-12-30,B19,Wind Onshore,Wind,NaN,UK,1.0,21658.0,MAW
21418,2022-12-30,2022-12-31,B19,Wind Onshore,Wind,NaN,UK,1.0,20614.0,MAW


In [18]:
df_gen_2022_daily.to_csv(f"/kaggle/working/gen_2022_daily.csv", index=False)

**Part 3:**

I resample data into 1 month interval from the 1st day of the month till last day of the month. So for one year, if there is data for 12 months then after the execution of the following code, 12 rows are generated where the start time is the start time of the 1st day of the month and the end time is the 24-hour day end time of the last day of the month.

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
file_name = "uk/gen_UK_B19.csv"

In [ ]:
df_gen = pd.read_csv(f"/kaggle/input/europe-green-electricity-generation-consumption/2022-2023/{file_name}")
df_gen

In [ ]:
from datetime import datetime, timedelta, timezone
start_date = datetime.fromisoformat(str(df_gen.loc[0, "StartTime"]).replace("Z", ""))
if start_date.year == 2021:
    df_gen = df_gen.drop(0)
    df_gen = df_gen.set_index(pd.Index(np.arange(0, df_gen.shape[0])))
df_gen

* I sum all values of quantiy column for the specific month to calculate the total energy generation for that month.

* I add few new columns in the resampled dataset, the new columns are:
1. AreaID
2. PsrType
3. Country
4. CountryID
5. UnitName
6. GenType
7. PsrTypeName

PsrType: This is the energy generation type, there are 9 types of green energy generation, they are:

* B01 = Biomass
* B09 = Geothermal
* B10 = Hydro Pumped Storage
* B11 = Hydro Run-of-river and poundage
* B12 = Hydro Water Reservoir
* B13 = Marine
* B15 = Other renewable
* B16 = Solar
* B18 = Wind Offshore
* B19 = Wind Onshore

GenType Energy generation type which is one level up of the PsrType.

* B18 and B19 = Wind
* B16 = Solar
* B10, B11, B12, B13 = Water
* B01, B09, B15 = Other

AreaID: the areas of the energy generation.

Country: CountryID:

* SP: 0, # Spain
* UK: 1, # United Kingdom
* DE: 2, # Germany
* DK: 3, # Denmark
* HU: 5, # Hungary
* SE: 4, # Sweden
* IT: 6, # Italy
* PO: 7, # Poland
* NL: 8 # Netherlands

Unit Name: MAW(Magnetostrictive Amorphous Wire)

In [ ]:
from datetime import datetime, timedelta, timezone
import calendar
country = "UK"
country_id = 1
#df_gen = df_gen.loc[1:]
file_exist = is_file_exist("/kaggle/input/europe-green-electricity-generation-consumption", "gen_2022_monthly.csv")
if file_exist == True:
    df_gen_2022_monthly = pd.read_csv("/kaggle/input/europe-green-electricity-generation-consumption/gen_2022_monthly.csv")
    indx = df_gen_2022_monthly.shape[0]
else:
    df_gen_2022_monthly = pd.DataFrame(data=[], columns=["StartTime", "EndTime", "PsrType", "PsrTypeName", "GenType", "AreaID", "Country", "CountryID", "quantity", "UnitName"])
    indx = 0
index = 0
init_date = datetime.fromisoformat(str(df_gen.loc[0, "StartTime"]).replace("Z", ""))
month = init_date.month
start_hour = init_date.hour
start_min = init_date.minute
quantity = 0
start_date = datetime.now()
pre_start_date = datetime.now()
area_id = ""
while index < df_gen.shape[0]:
    row = df_gen.loc[index]
    index = index + 1
    start_date = datetime.fromisoformat(str(row["StartTime"]).replace("Z", ""))
    if start_date.hour == start_hour and start_date.minute == start_min:
        area_id = row["AreaID"]
    if start_date.month == month:
        quantity = quantity + row["quantity"]
    else:
        pre_start_date = datetime.fromisoformat(str(df_gen.loc[index - 2, "StartTime"]).replace("Z", ""))
        pre_start_date = datetime(pre_start_date.year, pre_start_date.month, 1, tzinfo=start_date.tzinfo)
        df_gen_2022_monthly.loc[indx, "StartTime"] = pre_start_date
        days_in_month = calendar.monthrange(pre_start_date.year, pre_start_date.month)[1]
        interval_month = timedelta(days=days_in_month - 1)
        df_gen_2022_monthly.loc[indx, "EndTime"] = pre_start_date + interval_month
        df_gen_2022_monthly.loc[indx, "PsrType"] = row["PsrType"]
        df_gen_2022_monthly.loc[indx, "AreaID"] = area_id
        df_gen_2022_monthly.loc[indx, "Country"] = country
        df_gen_2022_monthly.loc[indx, "CountryID"] = country_id
        df_gen_2022_monthly.loc[indx, "quantity"] = quantity
        df_gen_2022_monthly.loc[indx, "UnitName"] = row["UnitName"]
        
        if row["PsrType"] == "B18" or row["PsrType"] == "B19":
            df_gen_2022_monthly.loc[indx, "GenType"] = "Wind"
        elif row["PsrType"] == "B16":
            df_gen_2022_monthly.loc[indx, "GenType"] = "Solar"
        elif row["PsrType"] == "B10" or row["PsrType"] == "B11" or row["PsrType"] == "B12" or row["PsrType"] == "B13":
            df_gen_2022_monthly.loc[indx, "GenType"] = "Water"
        elif row["PsrType"] == "B01" or row["PsrType"] == "B09" or row["PsrType"] == "B15":
            df_gen_2022_monthly.loc[indx, "GenType"] = "Other"

        if row["PsrType"] == "B18":
            df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Wind Offshore"
        elif row["PsrType"] == "B19":
            df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Wind Onshore"
        elif row["PsrType"] == "B16":
            df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Solar"
        elif row["PsrType"] == "B10":
            df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Hydro Pumped Storage"
        elif row["PsrType"] == "B11":
            df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Hydro Run-of-river and poundage"
        elif row["PsrType"] == "B12":
            df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Hydro Water Reservoir"
        elif row["PsrType"] == "B01":
            df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Biomass"
        elif row["PsrType"] == "B09":
            df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Geothermal"
        elif row["PsrType"] == "B13":
            df_gen_2022_daily.loc[indx, "PsrTypeName"] = "Marine" 
        elif row["PsrType"] == "B15":
            df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Other renewable"
        
        indx = indx + 1
        month = start_date.month
        next_row = datetime.fromisoformat(str(df_gen.loc[index, "StartTime"]).replace("Z", ""))
        start_hour = next_row.hour
        start_min = next_row.minute
        quantity = row["quantity"]

In [ ]:
pre_start_date = datetime.fromisoformat(str(df_gen.loc[index - 2, "StartTime"]).replace("Z", ""))
pre_start_date = datetime(pre_start_date.year, pre_start_date.month, 1, tzinfo=start_date.tzinfo)
df_gen_2022_monthly.loc[indx, "StartTime"] = pre_start_date
days_in_month = calendar.monthrange(pre_start_date.year, pre_start_date.month)[1]
interval_month = timedelta(days=days_in_month - 1)
df_gen_2022_monthly.loc[indx, "EndTime"] = pre_start_date + interval_month
df_gen_2022_monthly.loc[indx, "PsrType"] = row["PsrType"]
df_gen_2022_monthly.loc[indx, "AreaID"] = area_id
df_gen_2022_monthly.loc[indx, "Country"] = country
df_gen_2022_monthly.loc[indx, "CountryID"] = country_id
df_gen_2022_monthly.loc[indx, "quantity"] = quantity
df_gen_2022_monthly.loc[indx, "UnitName"] = row["UnitName"]

if row["PsrType"] == "B18" or row["PsrType"] == "B19":
    df_gen_2022_monthly.loc[indx, "GenType"] = "Wind"
elif row["PsrType"] == "B16":
    df_gen_2022_monthly.loc[indx, "GenType"] = "Solar"
elif row["PsrType"] == "B10" or row["PsrType"] == "B11" or row["PsrType"] == "B12" or row["PsrType"] == "B13":
    df_gen_2022_monthly.loc[indx, "GenType"] = "Water"
elif row["PsrType"] == "B01" or row["PsrType"] == "B09" or row["PsrType"] == "B15":
    df_gen_2022_monthly.loc[indx, "GenType"] = "Other"

if row["PsrType"] == "B18":
    df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Wind Offshore"
elif row["PsrType"] == "B19":
    df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Wind Onshore"
elif row["PsrType"] == "B16":
    df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Solar"
elif row["PsrType"] == "B10":
    df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Hydro Pumped Storage"
elif row["PsrType"] == "B11":
    df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Hydro Run-of-river and poundage"
elif row["PsrType"] == "B12":
    df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Hydro Water Reservoir"
elif row["PsrType"] == "B01":
    df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Biomass"
elif row["PsrType"] == "B09":
    df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Geothermal"
elif row["PsrType"] == "B13":
    df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Marine"    
elif row["PsrType"] == "B15":
    df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Other renewable"

In [ ]:
df_gen_2022_monthly

In [ ]:
df_gen_2022_monthly.to_csv(f"/kaggle/working/gen_2022_monthly.csv", index=False)